## Prepare

We allways need this chunck to use R in colab and mount google drive into the sesion

To use R magic in python colab. In 2023 the new rpy2 package was not able to run magic R, using a lower version soved the problem

In [1]:
!pip install rpy2==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp310-cp310-linux_x86_64.whl size=318085 sha256=03586e5e65c38b81b7bc98163b9cf7ca9f981b1fe253677e98edf2b372613688
  Stored in directory: /root/.cache/pip/wheels/73/a6/ff/4e75dd1ce1cfa2b9a670cbccf6a1e41c553199e9b25f05d953
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5


In [2]:
# activate R magic
%load_ext rpy2.ipython
# Load drive to use our files
from google.colab import drive
drive.mount('/content/drive')   # Change path

Mounted at /content/drive


## Definitions

In [3]:
%%R
# Parameters
MAX0 <- 0.10  # Keep genes with a certain number (perc.zero percent) of nonzero entries

# Directories
DATAdir <- "/content/drive/MyDrive/MELBOURNEbimodal/DATAinput/"   # Input files
TEMPdir <- "/content/drive/MyDrive/MELBOURNEbimodal/TEMPORAL/"  # Output temporal files
FINALdir <- "/content/drive/MyDrive/MELBOURNEbimodal/FINALdata/"  # Output of relevant files
IMAGEdir <- "/content/drive/MyDrive/MELBOURNEbimodal/IMAGESout/" # Image files





# Statements
set.seed(0xBEEF)
setwd(DATAdir)


# Useful function
`%ni%` <- Negate(`%in%`)

In [4]:
%%R
# Samples metadata: We have two kinds of data: Old data and New data
SAMPLESclass <- list()
SAMPLESclass[["NEW"]] <- data.frame(row.names = c("T0", "M15", "M05", "M02", "M10", "M11", "M07", "M12", "M09"), 
                                    TYPE = c("Baseline", "Vehicle", "Vehicle", "Mid Disease", "Mid Disease", "End Disease", "End Disease", "Post Disease", "Post Disease"),
                                    DAY = c(0, NA, NA, 2,2,5,5,7,7))

SAMPLESclass[["OLD"]] <- data.frame(row.names = c("T0", "M8", "M10"), TYPE = c("Vitro", "Vivo", "Vivo"))



Formating list of genes (20 lists based on G2M and Size-related genes)

Probided by Alex on Feb 23, 2023

In [5]:
%%R
FILE = readLines(paste(DATAdir, "genes_per_mouse_per_goal.txt", sep = ""))
GENESnewLists <- list()
for (LINE in FILE){
    if (grepl("=", LINE, fixed = TRUE)){
        
        LINE <- sub("= \\[", "<- c(", LINE)
        LINE <- sub("]", ")", LINE)
        print(LINE)
        GENESnewLists[[strsplit(LINE, " ")[[1]][1]]] <- eval(parse(text=LINE))
    }
}

[1] "g2m_d0p1 <- c('G2e3', 'Orc6', 'Calm2', 'Jpt1', 'Rnf214', 'Cbx5', 'Cdc20', 'Hells', 'Cenpf', 'Kif4', 'Atad2', 'Arhgap4', 'Ccne1', 'Psen1', 'Tubb5', 'Hmgb2', 'Arl6ip1', 'Kpna2', 'Hinfp', 'Ccne2', 'Plk1', 'Rpa2', 'Ccnb1', 'Rnf7', 'Ndc80', 'Rfc3', 'Top2a', 'Smc2', 'Kif11', 'Slbp', 'Cenpa', 'Cdk1', 'Cdca8', 'Kif20b', 'Birc5', 'Uhrf1', 'Sapcd2', 'Cdca3', 'Ube2s', 'Cdca7', 'Cenpl', 'Spc25', 'Mis18bp1', 'Depdc1a', 'Mcm3', 'Srp54a', 'Ppm1g', 'Mki67', 'Bub3', 'Aurka', 'Ubb', 'Tubb4b', 'Nme7', 'Atp11c', 'Cdkn2d', 'Malat1', 'Ube2c', 'Tpx2', 'Sgo2a', 'Prc1', 'Hist1h1b', 'Hmmr', 'Rangap1', 'Kif23', 'Parpbp', 'Cenpe', 'Cks2', 'Tuba1c', 'Ctcf', 'Pcna', 'Ckap2l', 'Cd47', 'Smc1a', 'Kifc1', 'Ung', 'Smc4', 'Aurkb', 'Nusap1')"
[1] "g2m_d0p2 <- c('G2e3', 'Orc6', 'Calm2', 'Jpt1', 'Cbx5', 'Cdc20', 'Hells', 'Cenpf', 'Pomgnt1', 'Nasp', 'Atad2', 'Ccne1', 'H2afx', 'Tubb5', 'Hmgb2', 'Arl6ip1', 'Kpna2', 'Cenpc1', 'Ccne2', 'Plk1', 'Rpa2', 'Ccnb1', 'Ogfod1', 'Ndc80', 'AI314180', 'Rfc3', 'Top2a', 'Smc2', 'Kif11',

In [6]:
%%R
# Relevant genes: First list
Sgenes <- c("Mcm5","Pcna","Tyms","Fen1","Mcm2","Mcm4","Rrm1","Ung","Gins2","Mcm6","Cdca7","Dtl","Prim1","Uhrf1","Mlf1ip","Hells","Rfc2","Rpa2","Nasp","Rad51ap1","Gmnn","Wdr76","Slbp","Ccne2","Ubr7","Pold3","Msh2","Atad2","Rad51","Rrm2","Cdc45","Cdc6","Exo1","Tipin","Dscc1","Blm","Casp8ap2","Usp1","Clspn","Pola1","Chaf1b","Brip1","E2f8")
G2Mgenes <- c("Hmgb2","Cdk1","Nusap1","Ube2c","Birc5","Tpx2","Top2a","Ndc80","Cks2","Nuf2","Cks1b","Mki67","Tmpo","Cenpf","Tacc3","Fam64a","Smc4","Ccnb2","Ckap2l","Ckap2","AurkB","Bub1","Kif11","Anp32e","Tubb4b","Gtse1","Kif20b","Hjurp","Cdca3","Hn1","Cdc20","Ttk","Cdc25c","Kif2c","Rangap1","Ncapd2","Dlgap5","Cdca2","Cdca8","Ect2","Kif23","Hmmr","Aurka","Psrc1","Anln","Lbr","Ckap5","Cenpe","Ctcf","Nek2","G2e3","Gas2l3","Cbx5","Cenpa")
GENES <- list(S = Sgenes, G2M = G2Mgenes)

# Ad it to the list of lists
GENESnewLists[["SvsG2M"]] <- unlist(GENES)

## Blank test

In [25]:
!ls ../FINALdata/
!pwd


FinalClassificationTable-All.tab
HEATMAPannot-ALLists-BimodalGenesNEW.Rds
HEATMAPannot-ALLists-scBimodalityClassGenesNEW.Rds
HEATMAPannot-ALLists-scClassGenesNEW.Rds
MeanSILHOUETTEcells-ALLists-BimodalGenesNEW.Rds
MeanSILHOUETTEcells-ALLists-scBimodalityClassGenesNEW.Rds
MeanSILHOUETTEcells-ALLists-scClassGenesNEW.Rds
/content/drive/MyDrive/MELBOURNEbimodal/DATAinput


In [38]:
%%R
#A <- readRDS("/content/drive/MyDrive/MELBOURNEbimodal/FINALdata/MeanSILHOUETTEcells-ALLists-scClassGenesNEW.Rds")
#print(A)
readRDS("/content/drive/MyDrive/MELBOURNEbimodal/FINALdata/MeanSILHOUETTEcells-ALLists-scBimodalityClassGenesNEW.Rds")

$g2m_d0p1
$g2m_d0p1$M15
$g2m_d0p1$M15$CELLS
$g2m_d0p1$M15$CELLS$NoG2M
[1] 0.4573063

$g2m_d0p1$M15$CELLS$G2M
[1] 0.8631287



$g2m_d0p1$M02
$g2m_d0p1$M02$CELLS
$g2m_d0p1$M02$CELLS$G2M
[1] 0.7196785

$g2m_d0p1$M02$CELLS$NoG2M
[1] 0.2971999



$g2m_d0p1$M12
$g2m_d0p1$M12$CELLS
$g2m_d0p1$M12$CELLS$NoG2M
[1] 0.3871608

$g2m_d0p1$M12$CELLS$G2M
[1] 0.8075952






In [31]:
%%R
for (i in names(A)){
    for (j in names(A[[i]])){
        str(lapply(A[[i]][[j]], function(x) lapply(x, mean)))
    }
}

List of 1
 $ CELLS:List of 2
  ..$ NoG2M: num 0.457
  ..$ G2M  : num 0.863
List of 1
 $ CELLS:List of 2
  ..$ G2M  : num 0.72
  ..$ NoG2M: num 0.297
List of 1
 $ CELLS:List of 2
  ..$ NoG2M: num 0.387
  ..$ G2M  : num 0.808
